<a href="https://colab.research.google.com/github/werowe/HypatiaAcademy/blob/master/spark/Imdb_spark_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

source:  https://datasets.imdbws.com/


In [1]:
!wget https://datasets.imdbws.com/title.ratings.tsv.gz

--2025-02-24 09:52:40--  https://datasets.imdbws.com/title.ratings.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 13.249.98.91, 13.249.98.35, 13.249.98.61, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|13.249.98.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7735068 (7.4M) [binary/octet-stream]
Saving to: ‘title.ratings.tsv.gz’

title.ratings.tsv.g 100%[===================>]   7.38M  21.7MB/s    in 0.3s    

2025-02-24 09:52:40 (21.7 MB/s) - ‘title.ratings.tsv.gz’ saved [7735068/7735068]



In [2]:
!gunzip title.ratings.tsv.gz

In [3]:
!head title.ratings.tsv

tconst	averageRating	numVotes
tt0000001	5.7	2133
tt0000002	5.5	289
tt0000003	6.4	2169
tt0000004	5.3	184
tt0000005	6.2	2897
tt0000006	5.0	208
tt0000007	5.3	901
tt0000008	5.4	2280
tt0000009	5.3	221


In [4]:
!head title.ratings.tsv > debug.tsv

In [5]:
!tail title.ratings.tsv  > debug.tsv

In [6]:
!cat title.ratings.tsv | egrep ".*([7-9])(\.).*" > good.rating.tsv


In [7]:
! wc -l *.tsv

      10 debug.tsv
  858793 good.rating.tsv
 1537374 title.ratings.tsv
 2396177 total


In [8]:
!head good.rating.tsv

tt0000012	7.4	13375
tt0000014	7.1	6103
tt0000060	7.1	93
tt0000211	7.4	5017
tt0000310	7.4	36
tt0000359	7.1	3356
tt0000361	7.4	27
tt0000417	8.1	58011
tt0000439	7.2	21753
tt0000441	7.2	12


In [9]:
from pyspark.sql.functions import col
from pyspark.sql import SparkSession

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DecimalType

# tconst	averageRating	numVotes
schema = StructType([
    StructField("tconst", StringType(), True),
    StructField("averageRating", DecimalType(4, 2), True),
    StructField("numVotes", IntegerType(), True)
])

spark = SparkSession.builder.appName("IMDB Ratings").getOrCreate()


df = spark.read.csv('good.rating.tsv', sep='\t', header=True,  schema=schema)




In [10]:
df.show()

+---------+-------------+--------+
|   tconst|averageRating|numVotes|
+---------+-------------+--------+
|tt0000014|         7.10|    6103|
|tt0000060|         7.10|      93|
|tt0000211|         7.40|    5017|
|tt0000310|         7.40|      36|
|tt0000359|         7.10|    3356|
|tt0000361|         7.40|      27|
|tt0000417|         8.10|   58011|
|tt0000439|         7.20|   21753|
|tt0000441|         7.20|      12|
|tt0000465|         7.30|    1968|
|tt0000499|         7.50|    4213|
|tt0000578|         7.20|      29|
|tt0000606|         7.40|      18|
|tt0000618|         8.90|      16|
|tt0000650|         7.40|      23|
|tt0000661|         7.80|      24|
|tt0000738|         7.90|      22|
|tt0000746|         7.80|      19|
|tt0000775|         7.50|      11|
|tt0000778|         7.00|      16|
+---------+-------------+--------+
only showing top 20 rows



In [11]:
#good_titles = df.filter(col('averageRating') >= 7.0)

#good_titles.show()

#save the filtered data to a new file, overwrite if the file already exists
#good_titles.write.csv('good.tsv', sep='\t', header=True, mode='overwrite')

In [12]:

#from pyspark.sql import SparkSession

# Initialize SparkSession
#spark = SparkSession.builder \
#    .appName("netflix") \
#    .getOrCreate()

##df = spark.read.option("ratings", "\t").option("header", True).csv("title.ratings.tsv")

# Display the DataFrame


#df = spark.read.csv(
#    "good.tsv",
#    header=True,        # Use the first row as column names
#    inferSchema=True,   # Automatically infer data types
#    sep="\t",            # Specify delimiter (default is ',')
#    encoding="UTF-8"    # Handle encoding
#)

#df.show()

In [13]:
!head title.ratings.tsv

tconst	averageRating	numVotes
tt0000001	5.7	2133
tt0000002	5.5	289
tt0000003	6.4	2169
tt0000004	5.3	184
tt0000005	6.2	2897
tt0000006	5.0	208
tt0000007	5.3	901
tt0000008	5.4	2280
tt0000009	5.3	221


In [14]:
df.createOrReplaceTempView("ratings")

In [15]:

sql = '''
describe ratings

'''

result = spark.sql(sql)
result.show()


+-------------+------------+-------+
|     col_name|   data_type|comment|
+-------------+------------+-------+
|       tconst|      string|   NULL|
|averageRating|decimal(4,2)|   NULL|
|     numVotes|         int|   NULL|
+-------------+------------+-------+



In [16]:
sql = '''
select * from ratings

'''

result = spark.sql(sql)
result.show()

+---------+-------------+--------+
|   tconst|averageRating|numVotes|
+---------+-------------+--------+
|tt0000014|         7.10|    6103|
|tt0000060|         7.10|      93|
|tt0000211|         7.40|    5017|
|tt0000310|         7.40|      36|
|tt0000359|         7.10|    3356|
|tt0000361|         7.40|      27|
|tt0000417|         8.10|   58011|
|tt0000439|         7.20|   21753|
|tt0000441|         7.20|      12|
|tt0000465|         7.30|    1968|
|tt0000499|         7.50|    4213|
|tt0000578|         7.20|      29|
|tt0000606|         7.40|      18|
|tt0000618|         8.90|      16|
|tt0000650|         7.40|      23|
|tt0000661|         7.80|      24|
|tt0000738|         7.90|      22|
|tt0000746|         7.80|      19|
|tt0000775|         7.50|      11|
|tt0000778|         7.00|      16|
+---------+-------------+--------+
only showing top 20 rows



In [17]:
!wget https://datasets.imdbws.com/title.basics.tsv.gz

--2025-02-24 09:53:10--  https://datasets.imdbws.com/title.basics.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 99.84.160.48, 99.84.160.41, 99.84.160.94, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|99.84.160.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 202185834 (193M) [binary/octet-stream]
Saving to: ‘title.basics.tsv.gz’

title.basics.tsv.gz 100%[===================>] 192.82M   168MB/s    in 1.2s    

2025-02-24 09:53:11 (168 MB/s) - ‘title.basics.tsv.gz’ saved [202185834/202185834]



In [18]:
!gunzip title.basics.tsv.gz

In [19]:
!head title.basics.tsv

tconst	titleType	primaryTitle	originalTitle	isAdult	startYear	endYear	runtimeMinutes	genres
tt0000001	short	Carmencita	Carmencita	0	1894	\N	1	Documentary,Short
tt0000002	short	Le clown et ses chiens	Le clown et ses chiens	0	1892	\N	5	Animation,Short
tt0000003	short	Poor Pierrot	Pauvre Pierrot	0	1892	\N	5	Animation,Comedy,Romance
tt0000004	short	Un bon bock	Un bon bock	0	1892	\N	12	Animation,Short
tt0000005	short	Blacksmith Scene	Blacksmith Scene	0	1893	\N	1	Short
tt0000006	short	Chinese Opium Den	Chinese Opium Den	0	1894	\N	1	Short
tt0000007	short	Corbett and Courtney Before the Kinetograph	Corbett and Courtney Before the Kinetograph	0	1894	\N	1	Short,Sport
tt0000008	short	Edison Kinetoscopic Record of a Sneeze	Edison Kinetoscopic Record of a Sneeze	0	1894	\N	1	Documentary,Short
tt0000009	movie	Miss Jerry	Miss Jerry	0	1894	\N	45	Romance


In [20]:
basics = spark.read.csv(
    "title.basics.tsv",
    header=True,        # Use the first row as column names
    inferSchema=True,   # Automatically infer data types
    sep="\t",            # Specify delimiter (default is ',')
    encoding="UTF-8"    # Handle encoding
)

basics.show()

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|        Poor Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             5|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|              

In [21]:
basics.createOrReplaceTempView("basics")

In [22]:
sql = '''
select * from basics

'''

result = spark.sql(sql)
result.show()


+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|        Poor Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             5|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|              

In [34]:
movies = spark.sql("""
    SELECT b.originalTitle, b.tconst, b.startYear, r.averageRating, r.numVotes
    FROM basics b
    INNER JOIN ratings r
    ON b.tconst = r.tconst
    where b.startYear RLIKE '^[0-9]+$' and LENGTH(r.numVotes) > 3 and titleType NOT IN ('videoGame', 'short', 'video')
    order by b.startYear desc, r.averageRating desc
""")

In [32]:
movies.cache()

DataFrame[originalTitle: string, tconst: string, startYear: string, averageRating: decimal(4,2), numVotes: int]

In [36]:
movies.count()

50322

In [35]:
movies.show()

+--------------------+----------+---------+-------------+--------+
|       originalTitle|    tconst|startYear|averageRating|numVotes|
+--------------------+----------+---------+-------------+--------+
|Don't Look Down o...|tt35667354|     2025|         9.70|    9113|
|       Ex-Degenerate|tt32267664|     2025|         9.60|    3675|
|Looking Up Was Ti...|tt35868733|     2025|         9.60|    2898|
|            Betrayal|tt35838909|     2025|         9.60|    2793|
|         Strike Last|tt32269370|     2025|         9.50|    3126|
|Anamadheya Ashok ...|tt34442671|     2025|         9.50|    1097|
|I Suppose You Are...|tt35283832|     2025|         9.50|    6720|
|           Awakening|tt35678050|     2025|         9.50|    3341|
|                Fire|tt34604827|     2025|         9.40|   10165|
|              Sangee|tt33084735|     2025|         9.30|    1928|
|           Nesippaya|tt27190164|     2025|         9.30|    2105|
|      Brahma Anandam|tt32324361|     2025|         9.30|    1

In [30]:
spark.sql("""
    SELECT distinct titleType from basics
""").show()

+------------+
|   titleType|
+------------+
|    tvSeries|
|tvMiniSeries|
|     tvMovie|
|   tvEpisode|
|       movie|
|   tvSpecial|
|       video|
|   videoGame|
|     tvShort|
|       short|
|     tvPilot|
+------------+



In [27]:
'''

#create a new Spark session and name it MovieRatings
spark = SparkSession.builder.appName("MovieRatings").getOrCreate()

#reads the ratings tsv as a csv, use the first row as headers, inferSchema
#will automatically figure out what type of data is in each column, like numbers or text
#sep tells that the data is seperated by tabs
#UTF-8 will help deal with any special charachters without errors
ratings = spark.read.csv("title.ratings.tsv", header=True, inferSchema=True, sep="\t", encoding="UTF-8")

#the same thing as the line before, but basics tsv is being read instead
basics = spark.read.csv("title.akas.tsv", header=True, inferSchema=True, sep="\t", encoding="UTF-8")

#explode the knownForTitles column, split the rows since they ara a list
#basics_exploded = basics.withColumn("tconst", explode(basics["titleId"]))

#rename the columns
basics = basics.withColumnRenamed("titleId", "titleId_basics")
ratings = ratings.withColumnRenamed("tconst", "titleId_ratings")

#join
result = basics.join(ratings, basics["titleId_basics"] == ratings["titleId_ratings"], how="inner")

result.show()

'''


'\n\n#create a new Spark session and name it MovieRatings\nspark = SparkSession.builder.appName("MovieRatings").getOrCreate()\n\n#reads the ratings tsv as a csv, use the first row as headers, inferSchema\n#will automatically figure out what type of data is in each column, like numbers or text\n#sep tells that the data is seperated by tabs\n#UTF-8 will help deal with any special charachters without errors\nratings = spark.read.csv("title.ratings.tsv", header=True, inferSchema=True, sep="\t", encoding="UTF-8")\n\n#the same thing as the line before, but basics tsv is being read instead\nbasics = spark.read.csv("title.akas.tsv", header=True, inferSchema=True, sep="\t", encoding="UTF-8")\n\n#explode the knownForTitles column, split the rows since they ara a list\n#basics_exploded = basics.withColumn("tconst", explode(basics["titleId"]))\n\n#rename the columns\nbasics = basics.withColumnRenamed("titleId", "titleId_basics")\nratings = ratings.withColumnRenamed("tconst", "titleId_ratings")\n\n

In [28]:
spark.sql("DESCRIBE TABLE basics").show()

+--------------+---------+-------+
|      col_name|data_type|comment|
+--------------+---------+-------+
|        tconst|   string|   NULL|
|     titleType|   string|   NULL|
|  primaryTitle|   string|   NULL|
| originalTitle|   string|   NULL|
|       isAdult|   string|   NULL|
|     startYear|   string|   NULL|
|       endYear|   string|   NULL|
|runtimeMinutes|   string|   NULL|
|        genres|   string|   NULL|
+--------------+---------+-------+



In [29]:
spark.sql("DESCRIBE TABLE ratings").show()

+-------------+------------+-------+
|     col_name|   data_type|comment|
+-------------+------------+-------+
|       tconst|      string|   NULL|
|averageRating|decimal(4,2)|   NULL|
|     numVotes|         int|   NULL|
+-------------+------------+-------+

